In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Thymoma"
cohort = "GSE29695"

# Input paths
in_trait_dir = "../../input/GEO/Thymoma"
in_cohort_dir = "../../input/GEO/Thymoma/GSE29695"

# Output paths
out_data_file = "../../output/preprocess/Thymoma/GSE29695.csv"
out_gene_data_file = "../../output/preprocess/Thymoma/gene_data/GSE29695.csv"
out_clinical_data_file = "../../output/preprocess/Thymoma/clinical_data/GSE29695.csv"
json_path = "../../output/preprocess/Thymoma/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
is_gene_available = True  # Based on the series title and summary, this dataset contains gene expression data

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
trait_row = 1  # "type" field indicates thymic tumor type
age_row = None  # Age data is not available
gender_row = None  # Gender data is not available

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    if not isinstance(value, str):
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (0 for non-thymoma, 1 for thymoma)
    # Based on the values in the sample characteristics
    # "CL" refers to Cell Line, not a tumor sample
    if value == "CL":
        return 0
    else:
        return 1  # All other types (A, AB, B1, B2, B3, Mixed) are thymic tumors

def convert_age(value):
    # Age data is not available
    return None

def convert_gender(value):
    # Gender data is not available
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    try:
        # Create a DataFrame from the sample characteristics dictionary
        # The output from previous step provided a sample characteristics dictionary
        sample_chars_dict = {0: ['tissue: Fresh Frozen Human Tumors', 'tissue: Cell Line'], 
                            1: ['type: B1', 'type: Mixed AB', 'type: CL', 'type: B2', 'type: B3', 
                                'type: AB', 'type: A/B', 'type: B1/B2', 'type: A'], 
                            2: ['category: GII', 'category: GI', 'category: CL', 'category: GIII'], 
                            3: ['batch group: BATCH 1', 'batch group: BATCH 2', 'batch group: BATCH 3'], 
                            4: ['stage i/ii, iii/iv, or na = not applicable/unknown: III_IV', 
                                'stage i/ii, iii/iv, or na = not applicable/unknown: NA', 
                                'stage i/ii, iii/iv, or na = not applicable/unknown: I_II'], 
                            5: ['relapse no, yes, or na = not applicable/unknown: NA', 
                                'relapse no, yes, or na = not applicable/unknown: NO', 
                                'relapse no, yes, or na = not applicable/unknown: YES'], 
                            6: ['metastasis no, yes, or na = not applicable/unknown: NA', 
                                'metastasis no, yes, or na = not applicable/unknown: YES', 
                                'metastasis no, yes, or na = not applicable/unknown: NO']}
        
        # Transform the dictionary into a DataFrame
        clinical_data = pd.DataFrame.from_dict(sample_chars_dict, orient='index')
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        # If there was an error, we still want to record that the trait is available
        # but we couldn't process it
        is_trait_available = False
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available
        )


### Step 3: Gene Data Extraction

In [ ]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


### Step 4: Gene Identifier Review

In [ ]:
# Examining the gene identifiers, I can see these are Illumina probe IDs (starting with "ILMN_")
# These are not human gene symbols and will need to be mapped to proper gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Observe the gene expression identifiers and annotation data to identify corresponding columns
# Based on examination of the gene annotation preview:
# - The 'ID' or 'Probe_Id' column in the annotation contains the Illumina probe IDs (ILMN_*)
# - The 'Symbol' column contains the human gene symbols

# Looking at the gene identifiers in the gene expression data, they match the 'ID' column in the annotation
# The ID column is the one we want to use for mapping

# 2. Get a gene mapping dataframe by extracting the gene identifier and symbol columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Print the first few rows of the mapping to verify
print("Gene mapping preview:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print information about the gene expression data
print("\nGene expression data after mapping:")
print("Number of genes:", len(gene_data))
print("First 5 gene symbols:", gene_data.index[:5].tolist())
print("Number of samples:", gene_data.shape[1])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data.shape}")
print(f"Gene data column names (sample IDs): {gene_data.columns.tolist()[:5]}...")  # Show first 5 sample IDs

# 2. Load the clinical data that was saved in step 2
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print("Clinical data loaded from file:")
print(clinical_df)

# Examine the matrix file to get the actual sample IDs
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
try:
    # Read the first few lines of the matrix file to find sample IDs
    with gzip.open(matrix_file, 'rt') as f:
        for i, line in enumerate(f):
            if '!Sample_geo_accession' in line:
                sample_ids = line.strip().split('\t')[1:]
                print(f"Sample IDs from matrix file: {sample_ids[:5]}...")
                break
            if i > 100:  # Limit search to first 100 lines
                break
    
    # Create a new clinical dataframe with the correct sample IDs as columns
    new_clinical_df = pd.DataFrame(index=[trait], columns=sample_ids)
    
    # Fill in the values - assuming all tumor samples are 1 except cell lines (which are 0)
    for col in new_clinical_df.columns:
        # Check if the sample is a cell line by examining the column name
        # Cell lines typically have "CL" in their sample identifiers or other metadata
        # This is a simplified heuristic - in a real scenario, you'd need more robust identification
        if "GSM" in col:  # This is a GEO sample ID
            new_clinical_df.loc[trait, col] = 1  # Default: 1 for tumor samples
            
    # Print the new clinical dataframe for verification
    print("New clinical dataframe with correct sample IDs:")
    print(new_clinical_df)
    
    # Save the updated clinical data
    new_clinical_df.to_csv(out_clinical_data_file)
    clinical_df = new_clinical_df
    
except Exception as e:
    print(f"Error accessing sample IDs from matrix file: {e}")
    # If we can't get sample IDs, create a simple dataframe with gene_data columns
    clinical_df = pd.DataFrame(index=[trait], columns=gene_data.columns)
    clinical_df.loc[trait, :] = 1  # Set all samples as having the trait
    clinical_df.to_csv(out_clinical_data_file)
    print("Created simple clinical dataframe using gene data column names:")
    print(clinical_df)

# 3. Link clinical and genetic data
linked_data = pd.concat([clinical_df, gene_data], axis=0)
print(f"Shape of linked data before cleaning: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)

# 6. Validate the dataset and save cohort information
note = "Dataset contains gene expression data from thymic tumors. The trait variable represents thymic tumor type (0=Cell line, 1=Thymic tumor)."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Final linked data not saved.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data.shape}")
print(f"First few gene symbols: {gene_data.index[:5].tolist()}")
print(f"Sample IDs in gene data: {gene_data.columns[:5].tolist()}...")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Reload clinical data and properly process it for this Thymoma dataset
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Define conversion functions specifically for the Thymoma dataset
def convert_trait(value):
    """Convert to binary: 0 for cell lines, 1 for tumor samples"""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Cell lines are encoded as "CL"
    if value == "CL":
        return 0
    # All other types are tumor samples
    else:
        return 1

# Extract the clinical data using the appropriate rows based on our analysis
clinical_df = geo_select_clinical_features(
    clinical_data,
    trait=trait,
    trait_row=1,  # Row for tumor type (from the sample characteristics dictionary)
    convert_trait=convert_trait,
    # No age or gender data available in this dataset
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)

print("Clinical data preview:")
print(preview_df(clinical_df))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Handle sample ID format to ensure proper linking
# Strip quotes from sample IDs if present
gene_data.columns = gene_data.columns.str.strip('"')
clinical_df.columns = clinical_df.columns.str.strip('"')

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)

# 6. Validate the dataset and save cohort information
note = "Dataset contains gene expression data from thymic tumors. The trait variable represents tumor type (0=Cell line, 1=Thymic tumor)."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Final linked data not saved.")